In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, StringType, BooleanType, TimestampType, IntegerType, ShortType
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.functions import col, regexp_replace, sha2, when, lit, uuid, to_timestamp

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "chamados_hora"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_chamado_log"
TABELA_INVALIDOS_DESTINO = "tb_chamado_log_invalidos"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"
nome_tabela_invalidos = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_INVALIDOS_DESTINO}"

In [0]:
df_chamados_hora_bronze = spark.read.table(nome_tabela_origem)

max_dt_ingestao = (
    df_chamados_hora_bronze
    .agg(F.max(F.col("ingestion_timestamp")).alias("max_ts"))
    .first()["max_ts"]
)

df_filtrado = df_chamados_hora_bronze.filter(
    F.col("ingestion_timestamp") == F.lit(max_dt_ingestao)
)

FORMATO_DATA = "dd/MM/yyyy HH:mm:ss"

df_transformado = (
    df_filtrado
    .select(
        F.col("id_chamado").cast(LongType()).alias("cd_chamado"),
        F.col("id_cliente").alias("cd_cliente_raw"), 
        F.to_timestamp(
            F.regexp_replace(F.col("hora_abertura_chamado"), r'\s?[\W_]*s\s?', ' '),
            FORMATO_DATA
        ).cast(TimestampType()).alias("dh_abertura"),
        F.to_timestamp(
            F.regexp_replace(F.col("hora_inicio_atendimento"), r'\s?[\W_]*s\s?', ' '),
            FORMATO_DATA
        ).cast(TimestampType()).alias("dh_inicio"),
        F.to_timestamp(
            F.regexp_replace(F.col("hora_finalizacao_atendimento"), r'\s?[\W_]*s\s?', ' '),
            FORMATO_DATA
        ).cast(TimestampType()).alias("dh_fim"),
        F.col("ingestion_timestamp").alias("dt_ingestion"), 
        F.col("origem").alias("dc_origem")
    )
    .withColumn(
        "cd_cliente",
        F.sha2(F.col("cd_cliente_raw").cast(StringType()), 256) 
    )
    .drop("cd_cliente_raw")
    .dropDuplicates(["cd_chamado"])
)

df_validacao = (
    df_transformado
    .withColumn("flag_chamado_valido", F.col("cd_chamado").isNotNull())
    .withColumn("flag_cliente_valido", F.col("cd_cliente").isNotNull())
    .withColumn("flag_timestamps_validos", 
        F.col("dh_abertura").isNotNull() & 
        F.col("dh_inicio").isNotNull() & 
        F.col("dh_fim").isNotNull()
    )
    .withColumn("flag_qualidade",
        F.when(
            F.col("flag_chamado_valido") & 
            F.col("flag_cliente_valido") & 
            F.col("flag_timestamps_validos"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

df_validos = df_validacao.filter(F.col("flag_qualidade") == "OK")
df_invalidos = df_validacao.filter(F.col("flag_qualidade") == "ERRO")

df_silver = df_validos.select(
    "cd_chamado",
    "cd_cliente",
    "dh_abertura",
    "dh_inicio",
    "dh_fim",
    "dt_ingestion",
    "dc_origem"
)

In [0]:
delta_table = DeltaTable.forName(spark, nome_tabela_destino)

delta_table.alias("destino").merge(
    df_silver.alias("origem"),
    "destino.cd_chamado = origem.cd_chamado"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Tabela {nome_tabela_destino} atualizada com sucesso!")
print(f"Total de registros válidos processados: {df_silver.count()}")

df_invalidos.write.format("delta").mode("overwrite").saveAsTable(nome_tabela_invalidos)

print(f"✅ Tabela {nome_tabela_invalidos} atualizada para auditoria!")
print(f"Total de registros inválidos: {df_invalidos.count()}")